# Executing Matlab code using Octave through python

In [5]:
import subprocess

In [6]:
# Comando para ejecutar el script de Octave
# Asegúrate de que la ruta al script de Octave sea accesible desde donde estás ejecutando este script de Python
command = ["octave-cli", "hello_world.m"]

In [7]:

# Ejecuta el comando y captura la salida
result = subprocess.run(command, capture_output=True, text=True)

# Imprime la salida estándar (incluyendo "Hola Mundo")
print("Salida estándar:")
print(result.stdout)
# Imprime el error estándar si existe alguno
if result.stderr:
    print("Error estándar:")
    print(result.stderr)


Salida estándar:

Error estándar:
octave-cli: error while loading shared libraries: libGL.so.1: cannot open shared object file: No such file or directory

